In [1]:
#importing required Library
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
#reload pickle files
pickel_file = 'notMNIST.pickle'

#open pickle file using context manager

with open(pickel_file,'rb') as f:
    save = pickle.load(f)
    #print(save.keys())
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    
    #delete save to free memory
    del save  

In [3]:
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (5000, 28, 28) (5000,)
Validation set (2500, 28, 28) (2500,)
Test set (2500, 28, 28) (2500,)


In [4]:
tf.logging.set_verbosity(tf.logging.INFO)

In [5]:
def cnn_model_fn(features, labels, mode):
    #create an input layer
    # shape of input layer will be [batch_size, width, height, channels]
    #image is of size 28*28
    input_layer = tf.reshape(features["x"],[-1,28,28,1])
    
    # convolution layer 1
    # number of filter used = 32
    # filter size = 5 * 5
    # activation function relu
    # padding added = Yes to presever input image size
    # input shape =[batch_size, 28, 28, 1]
    # output size = [batch_size, 28, 28, 32]
    
    conv1  = tf.layers.conv2d(inputs=input_layer,
                             filters=32,
                             kernel_size=[5,5],
                             padding="same",
                             activation=tf.nn.relu)
    # Pooling layer
    # max pool = filter: 2*2 = stride: 2
    # Input Tensor Shape: [batch_size, 28, 28, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 32]
    
    pool1 = tf.layers.max_pooling2d(inputs=conv1,pool_size=[2,2],strides=2)
    
    # Convolutional Layer #2 and Pooling Layer #2
    # Input Tensor Shape: [batch_size, 14, 14, 32]
    # Output Tensor Shape:[batch_size, 14, 14, 64]
    conv2 = tf.layers.conv2d(
          inputs=pool1,
          filters=64,
          kernel_size=[5, 5],
          padding="same",
          activation=tf.nn.relu)
    # Input Tensor Shape: [batch_size, 14, 14, 64]
    # Output Tensor Shape: [batch_size, 7, 7, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    print("shape",pool2.shape)
    
    # Flatten tensor into a batch of vectors 
    pool2_flat = tf.reshape(pool2,[-1,7*7*64])
    
    # Dense Layer
    # Densely connected layer with 1024 neurons
    # Input Tensor Shape: [batch_size, 7 * 7 * 64]
    # Output Tensor Shape: [batch_size, 1024] 
    dense = tf.layers.dense(inputs =pool2_flat,units=1024,activation=tf.nn.relu)
    
    # drop out layer
    # rate=0.4 40% of features will be dropped randomly to avoid overfitting 
    # dropout must only be performed during traning phase 
    dropout = tf.layers.dropout(inputs=dense,rate=0.4,training=mode==tf.estimator.ModeKeys.TRAIN)
    
    # Logits layer
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 10]
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    # logit layer gives row value of prediction
    # create dict which will find max value and probablities 
    predictions = {
        "classes" : tf.argmax(logits,axis=1),
        "probabilities" : tf.nn.softmax(logits,name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # convert data into one-hot encoding
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    
    # cal loss function
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels,logits=logits)
    
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
             loss=loss,
             global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode = mode,loss = loss,train_op=train_op)
    
    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = { "accuracy": tf.metrics.accuracy(labels=labels,
                                                        predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode = mode,loss=loss,eval_metric_ops=eval_metric_ops)

In [6]:
def main(unused_argv):
    train_data = train_dataset # Returns np.array
    eval_data = valid_dataset # Returns np.array
    eval_labels = valid_labels
    
    # Create the Estimator
    mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir=r"C:\Users\rohishah\Desktop\image_classfifier")
    # Set up logging for predictions
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)
    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y = train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
    mnist_classifier.train(input_fn=train_input_fn,
                        steps=10000,
                        hooks=[logging_hook])
    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
          x={"x": eval_data},
          y=eval_labels,
          num_epochs=1,
          shuffle=False)
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

In [ ]:
if __name__ == "__main__":
    tf.app.run()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': None, '_model_dir': 'C:\\Users\\rohishah\\Desktop\\image_classfifier', '_num_ps_replicas': 0, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002B541285160>, '_global_id_in_cluster': 0, '_save_checkpoints_steps': None, '_num_worker_replicas': 1, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_master': '', '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_evaluation_master': '', '_log_step_count_steps': 100, '_service': None, '_task_type': 'worker', '_is_chief': True}
INFO:tensorflow:Calling model_fn.
shape (100, 7, 7, 64)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\rohishah\Desktop\i

INFO:tensorflow:step = 1, loss = 2.28975
INFO:tensorflow:probabilities = [[ 0.09403638  0.10067616  0.09989507  0.10318471  0.12360033  0.09604679
   0.08908119  0.09217833  0.09936012  0.10194074]
 [ 0.09492642  0.10337129  0.08670463  0.10566898  0.11745975  0.09620418
   0.09968536  0.09799431  0.09666911  0.10131592]
 [ 0.09310188  0.10464439  0.09995602  0.10712838  0.10812052  0.1120213
   0.100307    0.09032167  0.09760527  0.08679359]
 [ 0.08803356  0.12507518  0.08792009  0.106769    0.10439157  0.09701436
   0.09851746  0.0987106   0.09460838  0.09895983]
 [ 0.08161172  0.1026294   0.08401306  0.11298627  0.11726858  0.09029649
   0.10156883  0.10536906  0.10373945  0.10051719]
 [ 0.10283214  0.1029927   0.08241162  0.10630388  0.11850131  0.09713873
   0.08539993  0.08876544  0.11218174  0.10347258]
 [ 0.09018856  0.11227134  0.08980772  0.09823032  0.11774248  0.09211227
   0.08924565  0.09665291  0.10938261  0.10436615]
 [ 0.09313723  0.10179347  0.08915303  0.100224    0.

INFO:tensorflow:global_step/sec: 3.45526
INFO:tensorflow:probabilities = [[ 0.10322583  0.11530821  0.09024258  0.10133132  0.12559505  0.08794879
   0.08734473  0.09292394  0.09247274  0.10360677]
 [ 0.10149229  0.11508799  0.08799563  0.09715755  0.13028352  0.08406941
   0.09815832  0.09125615  0.09928516  0.09521406]
 [ 0.10245859  0.11906955  0.0872182   0.10040016  0.11738863  0.08705404
   0.11217801  0.09903315  0.08514007  0.09005957]
 [ 0.092176    0.11011409  0.10166653  0.10252505  0.11224164  0.1052379
   0.08683076  0.10199768  0.09574811  0.09146228]
 [ 0.10867933  0.09273993  0.09986769  0.10146938  0.10477708  0.08550505
   0.09373578  0.10665892  0.10989575  0.0966711 ]
 [ 0.09040357  0.10956199  0.10301971  0.09535316  0.10483237  0.10888834
   0.0956552   0.09395965  0.09776123  0.10056472]
 [ 0.09354429  0.09981026  0.08529615  0.11363263  0.11392784  0.10490897
   0.10194287  0.099874    0.09638192  0.09068103]
 [ 0.09812634  0.11555687  0.09499729  0.09689345  0.

INFO:tensorflow:step = 101, loss = 2.27756 (28.936 sec)
INFO:tensorflow:probabilities = [[ 0.09923796  0.10057932  0.08771023  0.09128546  0.11968961  0.10251012
   0.09937982  0.0978872   0.1028313   0.09888902]
 [ 0.10809839  0.09654774  0.08068245  0.1098692   0.10966454  0.09698892
   0.09867992  0.0982389   0.09560868  0.10562126]
 [ 0.09352437  0.12167765  0.08234002  0.10563082  0.12655124  0.09828849
   0.09049696  0.09505117  0.09096789  0.09547142]
 [ 0.09484462  0.09584274  0.09627017  0.10267123  0.10916328  0.10581201
   0.10600616  0.0951316   0.09811178  0.09614645]
 [ 0.1007256   0.11140937  0.10156152  0.08410642  0.11727183  0.10852381
   0.07795504  0.1116821   0.10056065  0.08620368]
 [ 0.08354804  0.11603767  0.09569613  0.11745171  0.09831129  0.09144548
   0.09274416  0.09983518  0.10507489  0.09985549]
 [ 0.09829481  0.12463744  0.08383139  0.0908046   0.11049628  0.10562862
   0.09900838  0.0894814   0.10550263  0.0923145 ]
 [ 0.09619148  0.12150603  0.09501177

INFO:tensorflow:global_step/sec: 3.32613
INFO:tensorflow:probabilities = [[ 0.10257567  0.1210587   0.09122386  0.09676842  0.10962678  0.09364442
   0.09268031  0.09944331  0.08705354  0.10592498]
 [ 0.10758102  0.10533292  0.110484    0.09178203  0.1039525   0.10559388
   0.09509413  0.09153689  0.0927967   0.09584594]
 [ 0.10196272  0.11078     0.08901918  0.10591803  0.09579633  0.10358653
   0.10090674  0.08832902  0.10622267  0.09747871]
 [ 0.08963743  0.1216731   0.08635379  0.09037798  0.11123395  0.11046369
   0.09152136  0.09664966  0.09650598  0.10558312]
 [ 0.08532967  0.10573524  0.09239906  0.09998054  0.12107775  0.1070375
   0.09482417  0.09635952  0.10619079  0.09106568]
 [ 0.09939035  0.1155139   0.08821546  0.10309766  0.11404273  0.08935189
   0.09442663  0.10218525  0.10388903  0.0898871 ]
 [ 0.09321833  0.1004581   0.09554534  0.09218951  0.10557935  0.1013154
   0.11270124  0.08802972  0.11407235  0.09689064]
 [ 0.09655859  0.10416256  0.09522584  0.09737636  0.1

...

INFO:tensorflow:step = 7501, loss = 0.713422 (28.622 sec)
INFO:tensorflow:probabilities = [[ 0.00015192  0.00571973  0.0011034   0.00058805  0.01823444  0.95760739
   0.00047577  0.0060966   0.00933616  0.00068656]
 [ 0.00777602  0.03729293  0.00496409  0.00125219  0.91776961  0.00861673
   0.00205156  0.01408246  0.00598639  0.00020783]
 [ 0.0015608   0.01440769  0.00325981  0.95937765  0.00073823  0.00029738
   0.01506269  0.0001745   0.00234377  0.00277764]
 [ 0.00201796  0.98758698  0.00002164  0.00300301  0.00449985  0.00019486
   0.00027465  0.00167841  0.00028674  0.00043588]
 [ 0.29511461  0.01207908  0.01402203  0.00952632  0.02158536  0.01909978
   0.00708011  0.00757154  0.24261262  0.37130862]
 [ 0.00091606  0.00581348  0.0058965   0.97271085  0.00006213  0.0002482
   0.00258562  0.00001996  0.00230881  0.00943848]
 [ 0.01312562  0.08912667  0.00067731  0.18904793  0.00611602  0.54636955
   0.00479416  0.11766894  0.024959    0.00811478]
 [ 0.00070068  0.00748733  0.0009842

INFO:tensorflow:global_step/sec: 3.45603
INFO:tensorflow:probabilities = [[ 0.00406233  0.0064513   0.00217783  0.00888074  0.00308081  0.01108987
   0.00026142  0.00960381  0.41532287  0.53906906]
 [ 0.01252381  0.00444057  0.0061831   0.00952505  0.00807013  0.00764318
   0.00386979  0.00431196  0.82968611  0.11374624]
 [ 0.00045085  0.00026092  0.00099087  0.00018643  0.00453964  0.00039325
   0.0000783   0.00008367  0.98145479  0.01156136]
 [ 0.001287    0.01982969  0.00817708  0.93575191  0.00512507  0.00111998
   0.00232712  0.00112191  0.02260896  0.00265133]
 [ 0.0001175   0.00043531  0.0018253   0.00089165  0.00124274  0.00132878
   0.00018179  0.00008444  0.95797884  0.03591359]
 [ 0.00608683  0.00342273  0.7218821   0.01738133  0.10989276  0.01069884
   0.07240679  0.01207464  0.04268624  0.00346764]
 [ 0.00897661  0.00129738  0.00025825  0.00295266  0.0000524   0.00035659
   0.00089353  0.00312248  0.02241796  0.95967209]
 [ 0.03042904  0.36998805  0.00177691  0.02511508  0

INFO:tensorflow:step = 7601, loss = 0.579878 (28.931 sec)
INFO:tensorflow:probabilities = [[ 0.00283297  0.00088468  0.02892818  0.00343285  0.00338088  0.00124251
   0.9549464   0.00277736  0.00122224  0.00035195]
 [ 0.00345754  0.20561627  0.00065298  0.00137254  0.74232715  0.01320197
   0.00467332  0.02237478  0.00617112  0.00015241]
 [ 0.0044093   0.00945844  0.00452608  0.00135301  0.00103951  0.0001631
   0.97886389  0.00001042  0.00011725  0.00005906]
 [ 0.05729141  0.00534311  0.00148896  0.00635373  0.00598619  0.00123232
   0.00306065  0.89195818  0.02211853  0.00516701]
 [ 0.00335738  0.00317527  0.00759182  0.00434935  0.00174595  0.00032341
   0.97390294  0.00065132  0.00333813  0.00156443]
 [ 0.00182021  0.00575025  0.00320505  0.00357243  0.00863899  0.00132575
   0.00144025  0.00061435  0.95654827  0.01708447]
 [ 0.00179718  0.00873854  0.00019652  0.00099455  0.12819679  0.84156984
   0.0006118   0.01761027  0.0002139   0.00007068]
 [ 0.00202118  0.00091084  0.0031326

INFO:tensorflow:global_step/sec: 3.0112
INFO:tensorflow:probabilities = [[ 0.00071976  0.00084544  0.44123298  0.00007326  0.54066098  0.00219496
   0.00856803  0.00250078  0.00317595  0.00002783]
 [ 0.08977915  0.04619862  0.00109052  0.02200116  0.00955201  0.02381025
   0.06679967  0.71460336  0.01582566  0.01033958]
 [ 0.0000221   0.00008153  0.97648376  0.00023974  0.00706655  0.00014474
   0.013314    0.00007395  0.00245485  0.00011884]
 [ 0.00657709  0.0242607   0.01115163  0.0091232   0.01778366  0.01303087
   0.00253666  0.00113564  0.44521052  0.46919006]
 [ 0.00102788  0.00475267  0.00223396  0.0017142   0.01147683  0.00195182
   0.00199033  0.00025121  0.94869596  0.02590516]
 [ 0.05874948  0.1297677   0.01643701  0.52849603  0.00692533  0.00368527
   0.02041258  0.02616186  0.17609179  0.03327299]
 [ 0.03789268  0.0070965   0.00633199  0.20138611  0.00066207  0.00109653
   0.01870649  0.6519261   0.06059006  0.01431151]
 [ 0.00239855  0.01415372  0.00585189  0.91941166  0.

INFO:tensorflow:step = 7701, loss = 0.629879 (33.224 sec)
